<a href="https://www.kaggle.com/code/edaaydinea/deep-learning-with-pytorch-siamese-network?scriptVersionId=123511218" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Deep Learning with PyTorch : Siamese Network

*Author: Eda AYDIN*

# Siamese Network

![siamese-network.png](https://miro.medium.com/v2/resize:fit:1100/format:webp/1*bJABur9wzFNACosQkim8kw.png)

[Eng]

A Siamese Network is a type of neural network architecture that is used for tasks that involve finding similarities or differences between two input samples. The network consists of two identical subnetworks that share the same set of weights and are trained simultaneously.

The basic idea behind a Siamese Network is to learn a similarity metric between two input samples. In other words, the network is trained to output a high value when the two input samples are similar and a low value when they are dissimilar. This makes it useful for a variety of applications, such as image or text similarity matching, face recognition, and signature verification.

One of main advantages of a Siamese Network is that it can be trained with very few examples, making it useful for applications where data is limited. Additionally, the shared weights between the two subnetworks allow the model to generalize well to new inputs.

Siamese Networks have been shown to be effective in a wide range of applications, including image recognition, classification, and speech recognition. They have also been applied to natural language processing tasks such as sentence similarity and paraphrase detection.


[Tr]

Siamese Network, iki giriş örneği arasındaki benzerlik ve farklılıkları bulmak için kullanılan bir sinir ağı mimarisidir. Ağ, iki aynı alt ağdan oluşur ve her biri aynı ağırlık kümesini paylaşır.

Siamese Network'ün temel fikri, iki giriş örneği arasındaki benzerliği öğrenmektir. Bu nedenle, ağ, iki giriş örneği benzer olduğunda yüksek bir çıktı değeri verir ve farklı olduğunda ise düşük bir çıktı değeri verir. Bu, örneğin görüntü veya metin benzerliği eşleştirme, yüz tanıma veya imza doğrulama gibi birçok uygulama için kullanışlıdır.

Siamese Network'ü kullanmanın en büyük avantajı, sınırlı veriyle bile eğitilebilmesidir. Ayrıca, alt ağlar arasındaki ağırlık paylaşımı, modelin yeni girişlere iyi genelleme yapabilmesine olanak tanır.

Siamese Network, görüntü tanıma, sınıflandırma ve konuşma tanıma gibi birçok alanda etkili olduğu kanıtlanmıştır. Ayrıca, cümle benzerliği ve paraphrase tespiti gibi doğal dil işleme görevleri için de uygulanabilir.

# Download and Import libraries

In [ ]:
!pip instaLWP:Sll segmentation-models-pytorch -q
!pip install -U git+https://github.com/albumentations-team/albumentations -q
!pip install --upgrade opencv-contrib-python -q

In [ ]:
!git clone https://github.com/parth1620/Person-Re-Id-Dataset

In [ ]:
import sys
sys.path.append("/kaggle/working/Person-Re-Id-Dataset")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

"""
Timm: PyTorch Image Models (timm) is a library for state-of-the-art-image classification, containing a collection of image models, optimizers, schedulers, augmentations and much more.
"""
import timm

import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader

from skimage import io
from sklearn.model_selection import train_test_split

"""
tqdm is a library that is used for creating Python Progress Bars. It gets its name from the Arabic name taqaddum, which means 'progress. '
"""
from tqdm import tqdm

# Configurations

In [ ]:
DATA_DIR = "/kaggle/input/market-1501/Market-1501-v15.09.15/bounding_box_train/"
CSV_FILE = "/kaggle/working/Person-Re-Id-Dataset/train.csv"

BATCH_SIZE = 32
LR = 0.001
EPOCHS = 15

DEVICE = 'cuda'

In [ ]:
df = pd.read_csv(CSV_FILE)
df.head()

In [ ]:
row = df.iloc[11]



A_img = io.imread(DATA_DIR + row.Anchor)
P_img = io.imread(DATA_DIR + row.Positive)
N_img = io.imread(DATA_DIR + row.Negative)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (10,5))

ax1.set_title("Anchor")
ax1.imshow(A_img)

ax2.set_title("Positive")
ax2.imshow(P_img)

ax3.set_title("Negative")
ax3.imshow(N_img)

In [ ]:
train_df, valid_df = train_test_split(df, test_size = 0.20, random_state = 42)

# Create APN Dataset

In [ ]:
class APN_Dataset(Dataset):
    
    def __init__(self, df):
        self.df = df
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        row = self.df.iloc[idx]
        
        A_img = io.imread(DATA_DIR + row.Anchor)
        P_img = io.imread(DATA_DIR + row.Positive)
        N_img = io.imread(DATA_DIR + row.Negative)
        
        A_img = torch.from_numpy(A_img).permute(2, 0 ,1) / 255.0
        P_img = torch.from_numpy(P_img).permute(2, 0 ,1) / 255.0
        N_img = torch.from_numpy(N_img).permute(2, 0 ,1) / 255.0
        
        return A_img, P_img, N_img
        

In [ ]:
trainset = APN_Dataset(train_df)
validset = APN_Dataset(valid_df)

print(f"Size of trainset : {len(trainset)}")
print(f"Size of validset : {len(validset)}")

In [ ]:
idx = 40
A,P,N = trainset[idx]

f, (ax1, ax2, ax3) = plt.subplots(1,3,figsize= (10,5))

ax1.set_title('Anchor')
ax1.imshow(A.numpy().transpose((1,2,0)), cmap = 'gray')

ax2.set_title('Positive')
ax2.imshow(P.numpy().transpose((1,2,0)), cmap = 'gray')

ax3.set_title('Negative')
ax3.imshow(N.numpy().transpose((1,2,0)), cmap = 'gray')

# Load Dataset into Batches

In [ ]:
trainloader = DataLoader(trainset, batch_size = BATCH_SIZE,shuffle = True)
validloader = DataLoader(validset, batch_size = BATCH_SIZE)

In [ ]:
print(f"No. of batches in trainloader : {len(trainloader)}")
print(f"No. of batches in validloader : {len(validloader)}")

In [ ]:
for A, P, N in trainloader:
    break;

print(f"One image batch shape : {A.shape}")

# Create Model

In [ ]:
class APN_Model(nn.Module):
    
    def __init__(self, emb_size = 512):
        super(APN_Model, self).__init__()
        
        self.efficientnet = timm.create_model('efficientnet_b0', pretrained=True)
        self.efficientnet.classifier = nn.Linear(in_features = self.efficientnet.classifier.in_features,
                                                out_features = emb_size)
        
    
    def forward(self, images):
        embeddings = self.efficientnet(images)
        return embeddings

In [ ]:
model = APN_Model()
model.to(DEVICE)

# Create Train and Eval Function

In [ ]:
def train_fn(model, dataloader, optimizer, criterion):
    model.train() # ON Dropout
    total_loss = 0.0 
    
    for A,P,N in tqdm(dataloader):
        A,P,N = A.to(DEVICE), P.to(DEVICE), N.to(DEVICE)
        
        A_embs = model(A)
        P_embs = model(P)
        N_embs = model(N)
        
        loss = criterion(A_embs, P_embs, N_embs)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
    return total_loss / len(dataloader)

In [ ]:
def eval_fn(model, dataloader, criterion):
    model.eval()  # OFF Dropout
    total_loss = 0.0 
    
    with torch.no_grad():
        for A,P,N in tqdm(dataloader):
            A,P,N = A.to(DEVICE), P.to(DEVICE), N.to(DEVICE)

            A_embs = model(A)
            P_embs = model(P)
            N_embs = model(N)

            loss = criterion(A_embs, P_embs, N_embs)

            total_loss += loss.item()
        
        return total_loss / len(dataloader)

In [ ]:
criterion = nn.TripletMarginLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = LR)

# Create Training Loop

In [ ]:
best_valid_loss = np.Inf

for i in range(EPOCHS):
    train_loss = train_fn(model, trainloader, optimizer, criterion)
    valid_loss = eval_fn(model, validloader, criterion)
    
    if valid_loss < best_valid_loss:
        torch.save(model.state_dict(), "best_model.pt")
        best_valid_loss = valid_loss
        print("SAVED_WEIGHT_SUCCESS")
    
    print(f"EPOCHS: {i+1} train_loss: {train_loss} valid_loss: {valid_loss}")

# Get Anchor Embeddings

In [ ]:
def get_encoding_csv(model, anc_img_names):
    anc_img_names_arr = np.array(anc_img_names)
    encodings = []
    
    model.eval()
    with torch.no_grad():
        for i in tqdm(anc_img_names_arr):
            A = io.imread(DATA_DIR + i)
            A = torch.from_numpy(A).permute(2, 0, 1) / 255.0
            A = A. to(DEVICE)
            A_enc = model(A.unsqueeze(0)) # c,h,w --> (1,c,h,w)
            encodings.append(A_enc.squeeze().cpu().detach().numpy())
        
        encodings = np.array(encodings)
        encodings = pd.DataFrame(encodings)
        df_enc = pd.concat([anc_img_names, encodings], axis=1)
    
    return df_enc

In [ ]:
model.load_state_dict(torch.load("best_model.pt"))
df_enc = get_encoding_csv(model, df["Anchor"])

In [ ]:
df_enc.to_csv("database.csv", index=False)
df_enc.head()

# Inference

In [ ]:
def euclidean_dist(img_enc, anc_enc_arr):
    dist = np.sqrt(np.dot(img_enc-anc_enc_arr, (img_enc - anc_enc_arr).T))
    return dist

In [ ]:
idx = 0
img_name = df_enc["Anchor"].iloc[idx]
img_path = DATA_DIR + img_name

img = io.imread(img_path)
img = torch.from_numpy(img).permute(2, 0, 1) / 255.0

model.eval()
with torch.no_grad():
    img = img.to(DEVICE)
    img_enc = model(img.unsqueeze(0))
    img_enc = img_enc.detach().cpu().numpy()

In [ ]:
anc_enc_arr = df_enc.iloc[:, 1:].to_numpy()
anc_img_names = df_enc["Anchor"]

In [ ]:
distance = []

for i in range(anc_enc_arr.shape[0]):
    dist = euclidean_dist(img_enc, anc_enc_arr[i : i+1, :])
    distance = np.append(distance, dist)

In [ ]:
closest_idx = np.argsort(distance)

In [ ]:
from utils import plot_closest_imgs

plot_closest_imgs(anc_img_names, DATA_DIR, img, img_path, closest_idx, distance, no_of_closest = 10);

# Resources

* [How to train your siamese neural network](https://towardsdatascience.com/how-to-train-your-siamese-neural-network-4c6da3259463)